In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os

from neuropacks import PVC11
from noise_correlations import (analysis,
                                discriminability,
                                null_models,
                                utils,
                                plot)
from pratplot import colorbar as pcb

%matplotlib inline

In [ ]:
# Load PVC11 dataset
base_path = '/Volumes/pss/data/pvc11/data/spikes_gratings'
data_path = os.path.join(base_path, 'data_monkey1_gratings.mat')
pack = PVC11(data_path)

In [ ]:
# Get response matrix for PVC11 dataset, and stimuli
X = pack.get_response_matrix(transform=None)
stimuli = pack.get_design_matrix(form='angle')
# Delete neurons that don't response at all to a stimulus
X = np.delete(X, utils.get_nonresponsive_for_stim(X, stimuli), axis=1)
# Get units that are overall "responsive"
responsive_units = utils.get_responsive_units(
    X, stimuli, aggregator=np.mean,
    peak_response=10.)
X = X[:, responsive_units]

In [ ]:
# Random state
rng = np.random.RandomState(2332)

In [ ]:
dims = np.array([2, 5, 10, 15])

In [ ]:
n_dims = dims.size
n_dimlets = 20
n_repeats = 1000
v_s_lfis = np.zeros((n_dims, n_dimlets * pack.n_stimuli, n_repeats))
v_s_sdkls = np.zeros_like(v_s_lfis)
v_r_lfis = np.zeros_like(v_s_lfis)
v_r_sdkls = np.zeros_like(v_s_lfis)
v_lfis = np.zeros((n_dims, n_dimlets * pack.n_stimuli))
v_sdkls = np.zeros_like(v_lfis)
units = {idx: np.zeros((n_dimlets * pack.n_stimuli, dim)) for idx, dim in enumerate(dims)}
stims = np.zeros((n_dims, n_dimlets * pack.n_stimuli, 2))

In [ ]:
# Calculate null measures across dims
for idx, dim in enumerate(dims):
    print(idx)
    v_s_lfis[idx], v_s_sdkls[idx], v_r_lfis[idx], v_r_sdkls[idx], \
    v_lfis[idx], v_sdkls[idx], units[idx], stims[idx] = \
        analysis.calculate_null_measures(
            X, stimuli,
            n_dim=dim,
            n_dimlets=n_dimlets,
            rng=rng,
            n_repeats=n_repeats,
            circular_stim=True)

In [ ]:
# Calculate percentiles
p_r_lfi = 100 * np.mean(v_lfis[..., np.newaxis] > v_r_lfis, axis=2)
p_r_sdkl = 100 * np.mean(v_sdkls[..., np.newaxis] > v_r_sdkls, axis=2)

In [ ]:
tuning_curves = utils.get_tuning_curve(X, stimuli)
preferred_stimuli = pack.get_angles()[np.argmax(tuning_curves, 1)]

In [ ]:
pair = 10
plt.hist(v_r_lfis[0, pair], bins=30, cumulative=True, histtype='step')
plt.axvline(v_lfis[0, pair])

In [ ]:
pair = 10
plt.hist(v_r_lfis[2, pair], bins=30, cumulative=True, histtype='step')
plt.axvline(v_lfis[2, pair])

In [ ]:
stims = np.tile(pack.get_angles(), 20)

In [ ]:
pstim_d2 = preferred_stimuli[units[0]]
diffs = (np.abs(pstim_d2[:, 1] - pstim_d2[:, 0]) - 180) % 180

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='polar')
ax.scatter(stims, diffs)

In [ ]:
plt.scatter(diffs, p_r_lfi[0], alpha=0.2)

In [ ]:
for split in np.array_split(p_r_lfi[0], 20):
    plt.plot(pack.get_angles(), split, color='k', alpha=0.5)

In [ ]:
pair = 13
plt.axvline(pstim_d2[pair * 12, 0])
plt.axvline(pstim_d2[pair * 12, 1])
plt.plot(pack.get_angles(), np.array_split(p_r_lfi[0], 20)[pair], color='k', alpha=0.5)

In [ ]:
pair = 6
plt.plot(pack.get_angles(), np.array_split(p_r_lfi[2], 20)[pair], color='k', alpha=0.5)

In [ ]:
pstim_d10 = preferred_stimuli[units[2]]

In [ ]:
plt.scatter(
    np.sum((np.abs(np.subtract.outer(pstim_d10, pstim_d10)[np.arange(240), :, np.arange(240), :]) - 180) % 180, axis=(1,2)) / (2 * 45),
    p_r_lfi[-1],
    alpha=0.25
)